In [ ]:
# Check first whether GPU is on

0. Prepare before training

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# path alias
!ln -s /content/gdrive/MyDrive/Colab_Notebooks /mydrive

In [ ]:
# check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
# Check CUDA version to bring the right cudnn file
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# prepare the proper cudnn file in your google drive, and unzip it
!tar -xzvf "/mydrive/yolov3/cudnn-11.3-linux-x64-v8.2.0.53.tgz" -C /usr/local

In [ ]:
# Check whether unzip was completed well
!cat /usr/local/cuda/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

1. Clone, configure & compile Darknet

In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Compile darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

2. Copy & customize configuration file

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3_training.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

3. Create ".names" and ".data" files

In [ ]:
!echo -e 'pedestrian\nperson on bike\nbike' > data/obj.names

# Must create the directory for backup before executing the code below
!echo -e 'classes= 3\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3/main/backup' > data/obj.data

4. Save "yolov3_training.cfg" and "obj.names" files in Goodgle Drive

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/main/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/main/classes.txt

5. Create a folder and unzip image dataset

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

6. Create train.txt file

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

7. Download pre-trained weights for the convolutional layers file

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

8. Train!

In [ ]:
# Initial execution : Transfer Learning with darknet53.conv.74 
!LD_LIBRARY_PATH=/usr/local/cuda/lib64/ ./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# Uncomment the code below, if the Google colab connection was out and you want to continue training from where the previous training was ended
# !LD_LIBRARY_PATH=/usr/local/cuda/lib64/ ./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/main/backup/yolov3_training_last.weights -dont_show